In [16]:
%matplotlib inline

from numpythia import Pythia, STATUS, HAS_END_VERTEX, ABS_PDG_ID
from pyjet import cluster
import matplotlib.pyplot as plt
import numpy as np
import ROOT
from ROOT import TLorentzVector
from ROOT import TDatabasePDG 
from ROOT import gRandom, TH1, TH1D, cout, TCanvas
ROOT.gStyle.SetOptStat('')


In [2]:
selection = ((STATUS == 1) & ~HAS_END_VERTEX &
 (ABS_PDG_ID != 12) & (ABS_PDG_ID != 14) & (ABS_PDG_ID != 16))  #removing neutrinos

In [3]:
params = {
 "Beams:frameType":  2,   
 "Beams:idA": 2212, 
 "Beams:idB": 11,
 "Beams:eA" : 250,
 "Beams:eB"  : 20,
 "WeakBosonExchange:ff2ff(t:gmZ)": "on",
 "PhaseSpace:Q2Min": 10,
 #"SpaceShower:dipoleRecoil": "on",
 "SpaceShower:pTmaxMatch": 2,
 "PDF:lepton": "off",
 "TimeShower:QEDshowerByL": "off",
}

In [ ]:
##PHOTOPRODUCTION

In [ ]:
params = {
 "HardQCD:all": "on",
 "Beams:eCM": 100,
 "Beams:idA": 11,
 "Beams:idB": 2212,
 "PDF:lepton2gamma": "on",
 "Photon:Q2max":1.0, 
 #"Photon:Wmin":10.0,
 #"PhaseSpace:pTHatMin":5.0,
 "PhotonParton:all": "on",
 #"Photon:ProcessType":0,
 "Main:timesAllowErrors":10000,
}


In [ ]:

#    // 4 = direct-direct.
#np.dtype([('E', 'f8'), ('px', 'f8'), ('py', 'f8'), ('pz', 'f8'), ('mass', 'f8'),
#          ('prodx', 'f8'), ('prody', 'f8'), ('prodz', 'f8'), ('prodt', 'f8'),
#          ('pdgid', 'i4'), ('status', 'i4')])

In [4]:
pythia = Pythia(params=params)


 *-------  PYTHIA Process Initialization  --------------------------*
 |                                                                  |
 | We collide p+ with e- at a CM energy of 1.414e+02 GeV            |
 |                                                                  |
 |------------------------------------------------------------------|
 |                                                    |             |
 | Subprocess                                    Code |   Estimated |
 |                                                    |    max (mb) |
 |                                                    |             |
 |------------------------------------------------------------------|
 |                                                    |             |
 | f f' -> f f' (t-channel gamma*/Z0)             211 |   1.896e-04 |
 |                                                                  |
 *-------  End PYTHIA Process Initialization -----------------------*

 *-------  PYTHIA 

In [50]:
hpt = ROOT.TH1F("hpt","hpt",20,0,10.0)
heta = ROOT.TH1F("heta","heta",20,-8.0,8.0)
h_nch = ROOT.TH1F("h_nch","h_nch",20,0,20)
h_jetpt = ROOT.TH1F("h_jetpt","h_jetpt", 20, 0, 20)
h_jetmass = ROOT.TH1F("h_jetmass", "h_jetmass", 20,0,20)

h_Q2 = ROOT.TH1F("h_Q2","h_Q2",100,0.0,50.0)
h_W  = ROOT.TH1F("h_W", "h_W", 100, 0.0, 100.0)
h_leptonpt = ROOT.TH1F("h_leptonpt","h_leptonpt", 100, 0.0, 50.0)
h_leptonphi = ROOT.TH1F("h_leptonphi","h_leptonphi", 100, -np.pi, np.pi)

h_photonphi = ROOT.TH1F("h_photonphi","h_photonphi", 100, -np.pi, np.pi)
h_jetphi = ROOT.TH1F("h_jetphi","h_jetphi", 100, -np.pi, np.pi)


h_dphi = ROOT.TH1F("h_dphi","h_dphi", 100, 0, np.pi)

hpt.Sumw2()
heta.Sumw2()
h_nch.Sumw2()
h_jetpt.Sumw2()
h_jetmass.Sumw2()
h_leptonpt.Sumw2()
h_leptonphi.Sumw2()

Warning in <TROOT::Append>: Replacing existing TH1: hpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: heta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_nch (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_jetpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_jetmass (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_Q2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_W (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_leptonpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_leptonphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_dphi (Potential memory leak).


In [68]:
for event in pythia(events=1e4):
    if not event: continue
    #need to find proton, and incoming and outgoing electron to get the DIS variables: Q2, W, Xbj, y
    #incoming and outgoing electrons
    jets = cluster(event.all(selection &(ABS_PDG_ID != 11)), R=1, p=-1, ep=True).inclusive_jets(ptmin=5)
    if(len(jets)<1): continue
    
    
    for i in event.all():
        if i['pdgid'] == 2212 and i['status'] == 4:
            #print ' proton'
            #print i
            mass = ROOT.TDatabasePDG.Instance().GetParticle(2212).Mass()
            energy = np.sqrt( i['px']*i['px']+ i['py']*i['py']+ i['pz']*i['pz'] + mass*mass)
            v_proton = ROOT.TLorentzVector()
            v_proton.SetPxPyPzE(i['px'],i['py'],i['pz'],energy);
        elif i['pdgid']==11 and  i['status']==4:
            #print 'electron in'
            #print i 
            mass = ROOT.TDatabasePDG.Instance().GetParticle(11).Mass()
            energy = np.sqrt( i['px']*i['px']+ i['py']*i['py']+ i['pz']*i['pz'] + mass*mass)
            v_ein = ROOT.TLorentzVector()
            v_ein.SetPxPyPzE(i['px'],i['py'],i['pz'],energy);
            
        elif i['pdgid']==11 and  i['status']==1:
            #print 'electron out'
            #print i 
            mass = ROOT.TDatabasePDG.Instance().GetParticle(11).Mass()
            energy = np.sqrt( i['px']*i['px']+ i['py']*i['py']+ i['pz']*i['pz'] + mass*mass)
            v_eout = ROOT.TLorentzVector()
            v_eout.SetPxPyPzE(i['px'],i['py'],i['pz'],energy);
    #print  ' '

    #4 momentum of virtual photon: 
    v_photon = v_ein-v_eout
    
    #DIS VARIABLES:
    Q2  = -v_photon.M2()
    W2  = (v_proton+ v_photon).M2()
    x   = Q2/(2.*v_proton.Dot(v_photon))
    y   = (v_proton.Dot(v_photon)) / (v_proton.Dot(v_ein))
    
    #print 'Q2: ', Q2, ' W2: ', W2 , ' x:', x, ' y:', y  
    #print 'lepton pt and phi ' , v_eout.Pt(),  ' ' , v_eout.Phi()
    #print 'photon pt and phi ' , v_photon.Pt(),  ' ' , v_photon.Phi()
    #print 'Lepton-photon dphi ', np.abs(ROOT.TVector2.Phi_mpi_pi(v_eout.Phi()-v_photon.Phi()))
    h_Q2.Fill(Q2)
    h_W.Fill(np.sqrt(W2))
    h_leptonpt.Fill(v_eout.Pt())
    h_leptonphi.Fill(v_eout.Phi())
    h_photonphi.Fill(v_photon.Phi())
    

    
    #print('This events has # ', len(jets), 'jets')
    for jet in jets:
        h_jetpt.Fill(jet.pt)
        h_jetmass.Fill(jet.mass)
        #print ' jet' , jet.pt, ' ' , jet.eta,  ' phi ' , jet.phi, ' mass', jet.mass, ' #constituents: ' , len(jet)
        
        #jet-photon azimuthal difference.
        dphi = np.abs(ROOT.TVector2.Phi_mpi_pi(jet.phi-v_photon.Phi()))
        
        
        #print 'azimuthal difference ' , dphi
        h_dphi.Fill(dphi)
        
        h_jetphi.Fill(jet.phi)
        
        
        #look for constituents in jets: 
        #for constit in jet:
            
            #print constit
            
    #print '######################'   
    nch = 0
    for p in event.all(selection & (ABS_PDG_ID == 211)):
        pt = np.sqrt(p['px']*p['px']* + p['py']*p['py'])
        p3 = np.sqrt(p['px']*p['px']* + p['py']*p['py'] + p['pz']*p['pz'])
        eta = 0.5*np.log( (p3 + p['pz'])/(p3-p['pz']))
        if(pt<0.050): continue
        #print p['E'], ' ', p['pdgid'], ' ', pt
        hpt.Fill(pt) 
        heta.Fill(eta)
        nch +=1 
    h_nch.Fill(nch)
#print ' '
    


 Pythia::next(): 41000 events have been generated 
 PYTHIA Error in MiniStringFragmentation::fragment: no 1- or 2-body state found above mass threshold  
 PYTHIA Error in Pythia::next: hadronLevel failed; try again  

 Pythia::next(): 42000 events have been generated 

 Pythia::next(): 43000 events have been generated 

 Pythia::next(): 44000 events have been generated 

 Pythia::next(): 45000 events have been generated 

 Pythia::next(): 46000 events have been generated 

 Pythia::next(): 47000 events have been generated 

 Pythia::next(): 48000 events have been generated 

 Pythia::next(): 49000 events have been generated 

 Pythia::next(): 50000 events have been generated 

 *-------  PYTHIA Event and Cross Section Statistics  -------------------------------------------------------------*
 |                                                                                                                 |
 | Subprocess                                    Code |            Number of ev

In [70]:
c = ROOT.TCanvas('c','c',800,800)
h_nch.Draw()
c.Draw()
c.SaveAs("h_nch.pdf")
c.Clear()
heta.Draw()
c.SaveAs("h_eta.pdf")
c.Clear()
hpt.Draw()
#ROOT.gPad.SetLogy()
c.SaveAs("h_pt.pdf")
c.Clear()
h_jetpt.Draw()
c.SaveAs("h_jetpt.pdf")
c.Clear()
h_jetmass.Draw()
c.SaveAs("h_jetmass.pdf")
c.Clear()
h_Q2.Draw()
c.SaveAs("h_Q2.pdf")

c.Clear()
h_W.Draw()
c.SaveAs("h_W.pdf")

c.Clear()
h_leptonpt.Draw()
c.SaveAs('h_leptonpt.pdf')

c.Clear()
h_leptonphi.Draw()
c.SaveAs('h_leptonphi.pdf')

c.Clear()
h_dphi.Draw()
c.SaveAs("h_dphi.pdf")

c.Clear()
h_photonphi.Draw()
c.SaveAs("h_photonphi.pdf")

c.Clear()
h_jetphi.Draw()
c.SaveAs("h_jetphi")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file h_nch.pdf has been created
Info in <TCanvas::Print>: pdf file h_eta.pdf has been created
Info in <TCanvas::Print>: pdf file h_pt.pdf has been created
Info in <TCanvas::Print>: pdf file h_jetpt.pdf has been created
Info in <TCanvas::Print>: pdf file h_jetmass.pdf has been created
Info in <TCanvas::Print>: pdf file h_Q2.pdf has been created
Info in <TCanvas::Print>: pdf file h_W.pdf has been created
Info in <TCanvas::Print>: pdf file h_leptonpt.pdf has been created
Info in <TCanvas::Print>: pdf file h_leptonphi.pdf has been created
Info in <TCanvas::Print>: pdf file h_dphi.pdf has been created
Info in <TCanvas::Print>: pdf file h_photonphi.pdf has been created
Info in <TCanvas::Print>: ps file h_jetphi has been created
